# CSE 146 Lab 2: Causality
### ASSIGNED: January 24, 2020
### DUE: February 4, 2020

The purpose of this assignment is to familiarize yourself with concepts in causality, including the do-calculus and interventions, using interventions to reason about causal relationships in dat, and the difference between correlation and causation.

## Instructions 
Prior to beginning the assignment you should read the chapter Causality from _[Fair ML](https://fairmlbook.org/causal.html)_. In addition, if you're not familiar with treating [functions as objects in python](https://www.geeksforgeeks.org/passing-function-as-an-argument-in-python/)  or [basic probability](https://towardsdatascience.com/basic-probability-theory-and-statistics-3105ab637213) you should read the short articles linked. 

For each question, you should write python code to compute the answer to the questions that renders in a readable way according to the specifications of the question. You may only use the packages provided in the Background and Setup code. We will not be installing any packages during grading, and code that does not compile will negatively affect your grade.

This assignment is to be done <b>on your own</b>. You may discuss the assignment with your classmates, but do not share code or solutions. List the students you discussed the assignment with in the space provided below. You may visit any of the TAs in their office hours or ask questions on Piazza.

Be sure to rename this homework notebook so that it includes your last name. 

### List any students you talked with about this assignment here:
1. [person 1]
2. [person 2]
3. etc.

# Background and Setup

In this assignment we will be considering _structural causal models_ and _causal graphs_ describing the relationship between regular exercise, being overweight, and having heart disease, each represented by the variables X, W, and H, respectively. For each variable a 1 indicates a "yes" to the question and a 0 indicates a "no." For example, if I have the tuple (1,0,1), that corresponds to a person who exercises regularly, is not overweight, but does have heart disease. 

<b>Definition 1:</b> A _structural causal model,_ $\mathbb{M}$, is a set of variables $\{X_1,X_2,...,X_d\}$, where
\begin{equation}
X_i := f(Pa(X_i)) \text{ for } i = 1,2,...,d,
\end{equation}

$Pa(X_i) \subseteq \{X_1,X_2,...,X_d\} \cup U_i$ are the "parents" of $X_i$, and $U_1,U_2,...,U_d$ are jointly independent noise functions.

<b>Definition 2:</b> A _causal graph_ for $\mathbb{M}$, is a directed graph with nodes $\{X_1,X_2,...,X_d,U_1,U_2,...,U_d\}$ and directed edges such that there is an edge from $X_i \rightarrow X_j$ if and only if $X_i$ is a parent of $X_j$.

<b>Definition 3:</b> An _intervention_ on a variable, $X_i$, in a structural causal model, $\mathbb{M}$, is an assignment of a new distribution function $g$ to $X_i$, resulting in a new structural causal model, $\mathbb{M}[\text{do}(X_i := g)]$. Typically $g = c$, where $c$ is some constant.

<b>Definition 4:</b> The _expected value_ of a variable, $X$, is given by the formula
\begin{equation}
E(X) = \sum_x xP(x).
\end{equation}

Notice that if $P(x)$ is a uniform distribution computing the expected value of $X$ is equivalent to taking the average.

In [1]:
import pandas
import numpy
import random

def generate_dataset(n, generate_datapoint, X, W, H, columns = []):
    d = []
    for i in range(n):
        d.append(generate_datapoint(X, W, H))
        
    if len(columns) > 0:
        return pandas.DataFrame(d, columns = columns)
    else: 
        return pandas.DataFrame(d)

The function `generate_dataset` takes a length of the dataset, a function to generate a datapoint, three distribution functions to be passed to `generate_datapoint`, and an optional argument, columns. If you choose to rename the columns, the number of columns must match the size of the generated data point.

# Problems
### Problem 1 (15 points)
#### Question
Based on the code in `generate_M_datapoint` and in the distribution functions we have defined a _structural causal model,_ which we will call $\mathbb{M}$. 

(a) List the edges in the causal graph assosiated with $\mathbb{M}$ in the form $A \rightarrow B$, where $A$ is a parent of $B$. Be sure that the arrow is pointing in the right direction! (5 points)

(b) Analytically compute the probability that each variable in $\mathbb{M}$ is true (e.g. $P(X = 1)$) in our _structural causal model_ using the formula
\begin{equation}
P(A) = \sum_B P(A|B)P(B).
\end{equation}

Show your work! (<b>hint</b>: Start with $X$. Observe that `random.randint(a,b)` is a uniform distribution over the interval $[a,b]$.) (10 points)

In [2]:
def generate_M_datapoint(X, W, H):
    x = X()
    w = W(x)
    h = H(x)
    return [x, w, h]

def X_M_distribution():
    return random.randint(0,1)

def W_M_distribution(x):
    if(x == 1):
        return int(random.randint(0,5) < 1) #1/6
    else:
        return int(random.randint(0,2) < 1) #1/3

def H_M_distribution(x):
    if(x == 1):
        return int(random.randint(0,3) < 1) #1/4
    else:
        return int(random.randint(0,1) < 1) #1/2

#### Answer

(a) W <- X -> H

(b) P(X=1) = 1/2

    P(W=1) = P(W=1, X=1)*P(X=1) + P(W=1, X=0)*P(X=0) = 1/12 + 1/6 = 3/12
    
    P(H=1) = P(H=1, X=1)*P(X=1) + P(H=1, X=0)*P(X=0) = 1/8 + 1/4 = 3/8

### Problem 2 (10 points)
#### Question

(a) Using `generate_dataset`, generate a dataset called `M_data` of size 1,000,000 with column names 'X', 'W', and 'H' using the distributions defining $\mathbb{M}$ from Problem 1. (5 points)

(b) Using your new dataset, `M_data`, compute (and display) the conditional probability $P_\mathbb{M}( H = 1 | W = 1)$. (5 points)
#### Answer

In [3]:
# (a)
M_data = generate_dataset(1000000,generate_M_datapoint,X_M_distribution,W_M_distribution,H_M_distribution,['X','W','H'])

In [13]:
# (b)
W_rows = M_data[M_data['W'] == 1]
#H_rows = M_data[M_data['H'] == 1]
HW_rows = W_rows[W_rows['H'] == 1]
prob = (((len(HW_rows)/len(M_data))/(len(W_rows)/len(M_data))))
print(prob)

0.41810547086496697


### Problem 3 (20 points)
#### Question

(a) Using `generate_dataset`, generate a dataset named `MdoW1_data` of size 1,000,000 with column names 'X', 'W', and 'H' using the distributions defining $\mathbb{M}[\text{do}(W := 1)]$. Notice that this amounts to changing the distribution for $W$ meaning you will have to define a new distribution function for $W$ and a new function to generate a datapoint. Name the new data generation function for $\mathbb{M}[\text{do}(W := 1)]$ `generate_MdoW_datapoint` and your new function defining the distribution $W := 1$ `W_MdoW1_distribution`. (15 points)

(b) Using this dataset, compute (and display) $P_{\mathbb{M}[\text{do}(W := 1)]}(H =1)$. (5 points)

#### Answer

In [5]:
# (a)
def generate_MdoW_datapoint(X, W, H):
    x = X()
    w = W()
    h = H(x)
    return [x, w, h]

def W_MdoW1_distribution():
    return 1

MdoW1_data = generate_dataset(1000000,generate_MdoW_datapoint,X_M_distribution,W_MdoW1_distribution,H_M_distribution,['X','W','H'])

#MdoW1_data


In [14]:
# (b)
H1_rows = MdoW1_data[MdoW1_data['H'] == 1]
prob2 = ((len(H1_rows)/len(MdoW1_data)))
print(prob2)

0.374611


### Problem 4 (10 points + 2 extra credit)
#### Question
(a) List the edges in the causal graph assosiated with $\mathbb{M}[\text{do}(W := 1)]$ in the form $A \rightarrow B$, where $A$ is a parent of $B$. (5 points)

(b) How does $P_{\mathbb{M}[\text{do}(W := 1)]}(H =1)$ compare with the conditional probability $P_\mathbb{M}( H = 1 | W = 1)$ you computed in Problem 2? From this, what can you conclude about the causal relationship between $W$ and $H$ in $\mathbb{M}$? What is $X$ called in $\mathbb{M}$? (5 points)

(Extra Credit) What about the structure of the causal graph of $\mathbb{M}$ would tell us that $X$ is a ----. (2 points)

#### Answer

(a) X -> H

(b) prob = 0.41810547086496697 | prob2 = 0.374611, W and H are independent of each other. X is a confounding variable.

(Extra Credit)
If X was not a confounding variable, the two probabilities we calculated would be the same.


### Problem 5 (20 points)
#### Question
(a) Using `generate_dataset`, generate a dataset named `MdoW0_data` of size 1,000,000 with column names 'X', 'W', and 'H' using the distributions defining 𝕄[do(𝑊:=0)]. To do this, as before, define a new function for the distribution $W := 0$ named `W_MdoW0_distribution`. If you did Problem 3 well, you can reuse the function `generate_MdoW_datapoint`, otherwise you'll need to write a function `generate_MdoW0_datapoint`. (5 points)

(b) To further elucidate the relationship between $W$ and $H$ compute (and display)
\begin{equation}
E_{\mathbb{M}[\text{do}(W := 1)]}(H) - E_{\mathbb{M}[\text{do}(W := 0)]}(H). \text{ (10 points)}
\end{equation}

This is called the _average treatment effect_ of $W$ on $H$, and it tells us how much the intervention on $W$ changes the expectation of $H$.

(c) Does the insight from the _average treatment effect_ of $W$ on $H$ agree with your conclusions from Problem 4(b)? (5 points)


#### Answer

In [18]:
# (a)
def W_MdoW0_distribution():
    return 0

MdoW0_data = generate_dataset(1000000,generate_MdoW_datapoint,X_M_distribution,W_MdoW0_distribution,H_M_distribution,['X','W','H'])

#MdoW0_data

In [19]:
# (b)
H1_W0_rows = MdoW0_data[MdoW0_data['H'] == 1]
H1_W1_rows = MdoW1_data[MdoW1_data['H'] == 1]
ex_prob_h = ((len(H1_W1_rows))/(len(MdoW1_data))) - ((len(H1_W0_rows))/(len(MdoW0_data)))
print(ex_prob_h)

-0.0003309999999999702


(c) Yes because W should not influence the amount of times H = 1 in both datasets. 

### Problem 6 (25 points)
Up until now we have assumed that we know precisely the data generation process (enough to program it!), and so we have computed interventions in our _structural causal model_ by directly changing the data generation process and creating a new dataset. In the real world, it is unlikely that we would know exactly how data is generated, and it is even less likely that we would be able to gather a new dataset. However, given our expert knowledge of this domain, we can come up with a _causal graph_ that we believe represents how different columns in our dataset are causally dependent. From this, we can estimate the effect of an intervention using the formula
\begin{equation}
P_{\mathbb{M}[\text{do}(A := a)]}(B = b) = \sum_z P_{\mathbb{M}}(B = b | A = a, Pa(B) = z)P_{\mathbb{M}}(Pa(B) = z).
\end{equation}

#### Question
(a) What are the parent(s) of $W$ in $\mathbb{M}$, and what are the possible values they(it) can take? (5 points)

(b) Using the _causal graph_ you discovered in Problem 1(a) and the dataset `M_data` from Problem 2, compute (and display) $P_{\mathbb{M}[\text{do}(W := 1)]}(H = 1)$ with the above formula. (15 points)

(c) How does the estimated intervention compare to the intervention you computed in Problem 3(b)? What does this tell you about the _causal graph_ we've assumed? (5 points)


#### Answer

(a) The parent of W is X, in M

In [49]:
# (b)
x0_row = MdoW1_data[MdoW1_data['X'] == 0]
x1_row = MdoW1_data[MdoW1_data['X'] == 1]
w_row = MdoW1_data[MdoW1_data['W'] == 1]
h_row = w_row[w_row['H'] == 1]

#w_row_x0 = x0_row[x0_row['W'] == 1]
#w_row_x1 = x1_row[x1_row['W'] == 1]

h_given_wx0 = w_row_x0[w_row_x0['H'] == 1]
h_given_wx1 = w_row_x1[w_row_x1['H'] == 1]

x0_prob = ((len(x0_row)/len(MdoW1_data)))
x1_prob = ((len(x1_row)/len(MdoW1_data)))

#w_row_x0_prob = len(w_row_x0)/len(MdoW1_data)
#w_row_x1_prob = len(w_row_x1)/len(MdoW1_data)

#h_give_wx0_prob = len(h_given_wx0)/len(MdoW1_data)
#h_give_wx1_prob = len(h_given_wx1)/len(MdoW1_data)

hi = len(h_row)/len(MdoW1_data)
hi0 = len(w_row)/len(MdoW1_data)

h0_top = hi * hi0

h1_top = hi * hi0


cpr0 = h0_top / hi0

xo_cpr0 = cpr0 * x0_prob
x1_cpr1 = cpr0 * x1_prob


final0 = xo_cpr0 * x0_prob
final1 = x1_cpr1 * x1_prob

final = final0 + final1

print(final)

0.18730564373374542


(c) The intervention is about half of what we calculated in problem 3.

# References
[1] Barocas, Hardt, Narayanan: _Fairness and Machine Learning Ch. 4, Causality._ fairmlbook.org, http://www.fairmlbook.org, 2019.